In [1]:
sc

<SparkContext master=yarn appName=PySparkShell>

# Importing the Libraries

In [85]:
import pyspark

from pyspark import SparkConf,SparkContext
from pyspark.sql.functions import regexp_extract
from pyspark.sql.types import *
from pyspark.sql import SQLContext
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.sql.types import IntegerType
from pyspark.sql import Window
from pyspark.sql import DataFrameStatFunctions as statFunc


import pandas as pd
import sklearn.metrics as metrics
import numpy as np
from sklearn.neighbors import NearestNeighbors
from scipy.spatial.distance import correlation
from sklearn.metrics.pairwise import pairwise_distances
import ipywidgets as widgets
from IPython.display import display, clear_output
from contextlib import contextmanager
import warnings
warnings.filterwarnings('ignore')

from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.sql import Row

## Creating a spark session

In [92]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('COLLABORATIVE FILTERING USING THE NETFLIX DATA').getOrCreate()

## Creating the three dataframes for training, testing and movie data sets

In [4]:
from pyspark.sql.types import *

movies_df_schema = StructType(
  [StructField('movieId', IntegerType()),
   StructField('yearOfRelease', IntegerType()),
   StructField('title', StringType())]
)
Training_df_schema = StructType(
  [StructField('movieId', IntegerType()),
   StructField('userId', IntegerType()),
   StructField('ratings',DoubleType())]
)

Testing_df_schema = StructType(
  [StructField('movieId', IntegerType()),
   StructField('userId', IntegerType()),
   StructField('ratings',DoubleType())]
)

In [5]:
#Creating the training,testing and movie dataframes

df_training = sqlContext.read.format('txt').options(inferSchema=True).schema(Training_df_schema).csv('s3://netflixfinalvinay/TrainingRatings.txt')
 
df_testing = sqlContext.read.format('txt').options(inferSchema=True).schema(Testing_df_schema).csv('s3://netflixfinalvinay/TestingRatings.txt')
 
df_movies = sqlContext.read.format('txt').options(inferSchema=True).schema(movies_df_schema).csv('s3://netflixfinalvinay/movie_titles.txt')
 

In [6]:
#Count of each dataframes    

training_count = df_training.count()
testing_count = df_testing.count()
movies_count = df_movies.count()

In [7]:
#Printing the data and count
print('There are %s samples in training set , %s samples in testing set and %s samples in movies in the datasets' % (training_count,testing_count, movies_count))
print('Training:')
df_training.show(5)
print('Testing:')
df_testing.show(5)
print('Movies:')
df_movies.show(5,truncate=False)

There are 3255352 samples in training set , 100478 samples in testing set and 17770 samples in movies in the datasets
Training:
+-------+-------+-------+
|movieId| userId|ratings|
+-------+-------+-------+
|      8|1744889|    1.0|
|      8|1395430|    2.0|
|      8|1205593|    4.0|
|      8|1488844|    4.0|
|      8|1447354|    1.0|
+-------+-------+-------+
only showing top 5 rows

Testing:
+-------+-------+-------+
|movieId| userId|ratings|
+-------+-------+-------+
|      8| 573364|    1.0|
|      8|2149668|    3.0|
|      8|1089184|    3.0|
|      8|2465894|    3.0|
|      8| 534508|    1.0|
+-------+-------+-------+
only showing top 5 rows

Movies:
+-------+-------------+----------------------------+
|movieId|yearOfRelease|title                       |
+-------+-------------+----------------------------+
|1      |2003         |Dinosaur Planet             |
|2      |2004         |Isle of Man TT 2004 Review  |
|3      |1997         |Character                   |
|4      |1994      

## HOW MANY DISTINCT USERS AND DISTINCT ITEMS ARE THERE IN THE TEST SET ?

In [8]:
#finding out the distinct users and items/movies in the testing set

print("The distinct items in testing set are :",df_testing.select('movieID').distinct().count())
print("The distinct users in testing set are :",df_testing.select('userID').distinct().count())


The distinct items in testing set are : 1701
The distinct users in testing set are : 27555


In [9]:
#finding out the distinct users and items(movies) in the training set 

print("The distinct items in training set are :",df_training.select('movieID').distinct().count())
print("The distinct users in training set are :",df_training.select('userID').distinct().count())

The distinct items in training set are : 1821
The distinct users in training set are : 28978


# Finding Overall Rates for Users and Movies in both the sets 

## Some Initial analysis on training and testing sets 

In [10]:
#lets convert to a Pandas Dataframe 

pd_training = df_training.toPandas()

pd_testing = df_testing.toPandas()


In [87]:
#descrivbing the training dataset 

pd_training.describe()

,movieId,userId,ratings
count,3.255352e+06,3.255352e+06,3.255352e+06
mean,8.724660e+03,1.327058e+06,3.481188e+00
std,5.107402e+03,7.626887e+05,1.082873e+00
min,8.000000e+00,7.000000e+00,1.000000e+00
25%,3.893000e+03,6.716970e+05,3.000000e+00
50%,8.825000e+03,1.322467e+06,4.000000e+00
75%,1.332600e+04,1.988873e+06,4.000000e+00
max,1.774200e+04,2.649285e+06,5.000000e+00


In [88]:
pd_testing.describe()

,movieId,userId,ratings
count,100478.000000,1.004780e+05,100478.000000
mean,8701.547792,1.329956e+06,3.479458
std,5098.075495,7.625041e+05,1.085280
min,8.000000,7.000000e+00,1.000000
25%,3893.000000,6.774300e+05,3.000000
50%,8699.000000,1.325031e+06,4.000000
75%,13298.000000,1.995052e+06,4.000000
max,17742.000000,2.649285e+06,5.000000


In [11]:
pd_training.tail()

,movieId,userId,ratings
3255347,17742,46222,3.0
3255348,17742,2534701,1.0
3255349,17742,208724,3.0
3255350,17742,483107,2.0
3255351,17742,1181331,2.0


In [12]:
pd_testing.tail()

,movieId,userId,ratings
100473,17742,1898310,2.0
100474,17742,716096,4.0
100475,17742,38115,3.0
100476,17742,2646347,5.0
100477,17742,273576,2.0


## Testing for null values in both the sets

In [89]:
pd_training.isnull().sum()

movieId    0
userId     0
ratings    0
dtype: int64

In [90]:
pd_testing.isnull().sum()

movieId    0
userId     0
ratings    0
dtype: int64

In [13]:
print("The counts of both the dataframes are ","Training dataset ==> ",len(pd_training),"Testing dataset ==>",len(pd_testing))

The counts of both the dataframes are  Training dataset ==>  3255352 Testing dataset ==> 100478


# Average Movie Ratings

In [14]:
#checking overall average rate of the movies with their counts  in the training set

averageMovieRatingsTraining = pd.DataFrame(pd_training.groupby('movieId')['ratings'].mean())

averageMovieRatingsTraining['counts'] =pd.DataFrame(pd_training.groupby('movieId')['ratings'].count())

averageMovieRatingsTraining.head()

,ratings,counts
movieId,,
8,3.055104,2831
28,3.760127,12244
43,2.310345,58
48,3.620648,1666
61,2.385965,57


In [15]:
#checking overall average rate of the movies with their counts  in the testing set

averageMovieRatingsTesting = pd.DataFrame(pd_testing.groupby('movieId')['ratings'].mean())

averageMovieRatingsTesting['counts'] =pd.DataFrame(pd_testing.groupby('movieId')['ratings'].count())

averageMovieRatingsTesting.head()

,ratings,counts
movieId,,
8,3.072917,96
28,3.698667,375
43,4.000000,1
48,3.622642,53
61,4.000000,3


In [16]:
averageMovieRatingsTesting.sort_values(by=['counts'],ascending=False).head(10)

,ratings,counts
movieId,,
6971,4.123305,811
4640,4.006614,756
6287,3.716418,737
9728,3.807365,706
8915,4.007194,695
4432,3.674855,692
8596,4.091172,691
6408,3.876471,680
1406,3.233129,652


# Average User Ratings

In [17]:
#checking overall average rate of the user with their counts  in the training set

averageUserRatingsTraining = pd.DataFrame(pd_training.groupby('userId')['ratings'].mean())

averageUserRatingsTraining['counts'] =pd.DataFrame(pd_training.groupby('userId')['ratings'].count())

averageUserRatingsTraining.head()

,ratings,counts
userId,,
7,3.903846,104
79,3.630952,84
199,3.943662,71
481,4.351351,74
769,3.193878,98


In [18]:
#checking overall average rate of the user with their counts  in the testing set

averageUserRatingsTesting = pd.DataFrame(pd_testing.groupby('userId')['ratings'].mean())

averageUserRatingsTesting['counts'] =pd.DataFrame(pd_testing.groupby('userId')['ratings'].count())

averageUserRatingsTesting.tail()

,ratings,counts
userId,,
2648869,2.777778,9
2648885,4.000000,5
2649120,5.000000,2
2649267,4.142857,7
2649285,2.333333,3


In [19]:
averageUserRatingsTesting.sort_values(by=['counts'],ascending=False).head(10)

,ratings,counts
userId,,
1664010,4.300000,70
2439493,1.307692,52
305344,1.711538,52
387418,1.941176,51
1314869,3.263158,38
2118461,4.117647,34
1932594,2.321429,28
491531,2.629630,27
2606799,2.888889,27


## ESTIMATED AVERAGE OVERLAP OF ITEMS FOR USERS

In [20]:
#Pivoting the table with userID as index and the movieId as columns for our training data

userItemRating = pd_training.pivot_table(index='userId', columns='movieId', values='ratings')

userItemRating.head()

movieId,8,28,43,48,61,64,66,92,96,111,...,17654,17660,17689,17693,17706,17725,17728,17734,17741,17742
userId,,,,,,,,,,,,,,,,,,,,,
7,5.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
79,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
199,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
481,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
769,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
pd_testing.head()

,movieId,userId,ratings
0,8,573364,1.0
1,8,2149668,3.0
2,8,1089184,3.0
3,8,2465894,3.0
4,8,534508,1.0


In [22]:
userItemRating.fillna(0, inplace = True)

userMovieRating = userItemRating.astype(np.int32)

userMovieRating.head()

movieId,8,28,43,48,61,64,66,92,96,111,...,17654,17660,17689,17693,17706,17725,17728,17734,17741,17742
userId,,,,,,,,,,,,,,,,,,,,,
7,5,4,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
79,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
199,0,0,0,0,0,0,0,0,0,4,...,0,0,0,0,0,0,0,0,0,0
481,0,0,0,0,0,0,0,0,0,5,...,0,0,0,0,0,0,0,0,0,0
769,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0



## I am using the top 10 users and movies which are having the most counts from the test dataset.


In [ ]:
userMovieRating[17742].head(50)

In [23]:
def average(user):
    summed =0
    count =0

    for i in range(len(user)):
        if(user[i]==0):
            summed = summed +0
        else:
            summed= user[i] +summed
            count= 1+ count
    avgg=summed/count
    return avgg

In [24]:
pd_testing.tail()

,movieId,userId,ratings
100473,17742,1898310,2.0
100474,17742,716096,4.0
100475,17742,38115,3.0
100476,17742,2646347,5.0
100477,17742,273576,2.0


In [25]:
user1664010= list(userMovieRating.loc[1664010])
avg1664010= average(user1664010)
print(avg1664010)

4.2384364820846905


In [26]:
user2439493 = list(userMovieRating.loc[2439493])

avg2439493= average(user2439493)

print(avg2439493)

1.225609756097561


In [27]:
user305344 = list(userMovieRating.loc[305344])

avg305344 = average(user305344)

print(avg305344)

1.904382470119522


In [28]:
user387418 = list(userMovieRating.loc[387418])

avg387418= average(user387418)

print(avg387418)

1.8405963302752293


In [29]:
user1314869 = list(userMovieRating.loc[1314869])

avg1314869= average(user1314869)

print(avg1314869)

2.970984455958549


In [30]:
user2118461 = list(userMovieRating.loc[2118461])

avg2118461= average(user2118461)

print(avg2118461)

4.088453747467927


In [31]:
user1932594 = list(userMovieRating.loc[1932594])

avg1932594 = average(user1932594)

print(avg1932594)

2.3195754716981134


In [32]:
user491531 = list(userMovieRating.loc[491531])

avg491531= average(user491531)

print(avg491531)

2.9102384291725105


In [ ]:
user2606799 = list(userMovieRating.loc[2606799])

avg2606799 = average(user2606799)

print(avg2606799)

In [34]:
user727242 = list(userMovieRating.loc[727242])

avg727242 = average(user727242)

print(avg727242)

1.1890694239290989


In [35]:
ESTIMATED_AVERAGE_ITEMS_USERS = avg1314869+avg387418+avg305344+avg2439493+avg1664010+avg727242+avg2606799+avg491531+avg1932594+avg2118461

print("ESTIMATED AVERAGE OVERLAP OF ITEMS FOR USERS is : ",ESTIMATED_AVERAGE_ITEMS_USERS/10)

ESTIMATED AVERAGE OVERLAP OF ITEMS FOR USERS is :  2.550532902294355


### ESTIMATED AVERAGE OVERLAP OF ITEMS FOR USERS is :  2.550532902294355

## ESTIMATED AVERAGE OVERLAP OF USERS FOR ITEMS# 

In [36]:
#creating a utility matrix with movieId as index and the userId as columns to training data

movieUserRating = pd_training.pivot_table(index='movieId', columns='userId', values='ratings')

movieUserRating.head()

userId,7,79,199,481,769,906,1310,1333,1427,1442,...,2648572,2648589,2648730,2648734,2648853,2648869,2648885,2649120,2649267,2649285
movieId,,,,,,,,,,,,,,,,,,,,,
8,5.0,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
28,4.0,NaN,NaN,NaN,NaN,3.0,3.0,2.0,NaN,4.0,...,NaN,3.0,4.0,NaN,2.0,NaN,4.0,NaN,NaN,4.0
43,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
48,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
61,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [37]:
movieUserRating.fillna(0, inplace = True)

movieUserRating = movieUserRating.astype(np.int32)

movieUserRating.head()

userId,7,79,199,481,769,906,1310,1333,1427,1442,...,2648572,2648589,2648730,2648734,2648853,2648869,2648885,2649120,2649267,2649285
movieId,,,,,,,,,,,,,,,,,,,,,
8,5,0,0,0,0,0,0,3,0,0,...,0,0,0,0,0,0,0,0,0,0
28,4,0,0,0,0,3,3,2,0,4,...,0,3,4,0,2,0,4,0,0,4
43,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
48,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
61,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [38]:
movie6971= list(movieUserRating.loc[6971])

avg6971= average(movie6971)

print(avg6971)

4.071815611748076


In [46]:

movie4640= list(movieUserRating.loc[4640])

avg4640 = average(movie4640)

print(avg4640)

4.047438894792774


In [47]:

movie6287= list(movieUserRating.loc[6287])

avg6287= average(movie6287)

print(avg6287)

3.7261099495756977


In [45]:

movie9728  = list(movieUserRating.loc[9728])

avg9728= average(movie9728)

print(avg9728)

3.8246204278812974


In [44]:
8915
movie8915 = list(movieUserRating.loc[8915])

avg8915 = average(movie8915)

print(avg8915)

3.9663373050469826


In [43]:
#4432
movie4432= list(movieUserRating.loc[4432])

avg4432 = average(movie4432)

print(avg4432)

3.6721471305118545


In [42]:
#8596
movie8596= list(movieUserRating.loc[8596])

avg8596 = average(movie8596)

print(avg8596)

4.103368832862421


In [41]:
#6408
movie6408= list(movieUserRating.loc[6408])

avg6408 = average(movie6408)

print(avg6408)

3.8013020096235492


In [40]:

movie1406= list(movieUserRating.loc[1406])

avg1406 = average(movie1406)

print(avg1406)

3.2807910426057876


In [39]:
movie1744= list(movieUserRating.loc[1744])

avg1744 = average(movie1744)

print(avg1744)

3.802730883813307


In [52]:

ESTIMATED_AVERAGE_USER_ITEMS = avg6971+avg4640+avg6287+avg9728+avg8915+avg1744+avg1406+avg6408+avg8596+avg4432

print("ESTIMATED AVERAGE OVERLAP OF USERS FOR ITEMS : ",ESTIMATED_AVERAGE_USER_ITEMS/10)

ESTIMATED AVERAGE OVERLAP OF USERS FOR ITEMS :  3.8296662088461746


### ESTIMATED AVERAGE OVERLAP OF USERS FOR ITEMS :  3.8296662088461746

# ALS Model Approach

In [54]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.sql import Row

In [55]:
#Joining the actual dataframes with movie dataframe for accurate understandings

df_training_join = df_training.join(df_movies,on=['movieId'],how='inner')

df_testing_join = df_testing.join(df_movies,on=['movieId'],how='inner')

### Approach 1, maxIter = 5 , regParam = 0.08

In [101]:
als = ALS(maxIter=5, regParam=0.08, userCol="userId", itemCol="movieId", ratingCol="ratings",
          coldStartStrategy="drop")
model1 = als.fit(df_training_join)



# Evaluate the model by computing the RMSE on the test data
predictions1 = model1.transform(df_testing_join)
evaluator = RegressionEvaluator(metricName="mse", labelCol="ratings",
                                predictionCol="prediction")
                                            
evaluator1 = RegressionEvaluator(metricName="rmse", labelCol="ratings",
                                predictionCol="prediction")



rmse = evaluator1.evaluate(predictions1)
mse = evaluator.evaluate(predictions1)

print("Mean squared error = " + str(mse))
print("Root-mean-square error = " + str(rmse))

Mean squared error = 0.7396199565887807
Root-mean-square error = 0.860011602589628


### With the parameters maxIter = 5 , regParam = 0.08, Root-mean-square error is 0.860 and Mean squared error is 0.739

In [102]:
print("Following are the predictions results for the approach 1 model. \n")

predictions1.show()

Following are the predictions results for the approach 1 model. 

+-------+-------+-------+-------------+-------------+----------+
|movieId| userId|ratings|yearOfRelease|        title|prediction|
+-------+-------+-------+-------------+-------------+----------+
|   2366| 397212|    2.0|         2001|Just Visiting| 1.9941576|
|   2366| 692235|    3.0|         2001|Just Visiting| 2.9941514|
|   2366|1848661|    3.0|         2001|Just Visiting| 3.0412455|
|   2366|2522831|    3.0|         2001|Just Visiting| 2.5490205|
|   2366|1889646|    4.0|         2001|Just Visiting| 3.2688239|
|   2366| 971361|    1.0|         2001|Just Visiting| 1.9191813|
|   2366| 371750|    3.0|         2001|Just Visiting| 3.4278243|
|   2366|2392538|    3.0|         2001|Just Visiting| 2.8423905|
|   2366| 934835|    4.0|         2001|Just Visiting| 2.2479072|
|   2366|1967163|    3.0|         2001|Just Visiting| 2.8313835|
|   2366| 493190|    3.0|         2001|Just Visiting|  2.858516|
|   2366| 472083|    4.0

In [106]:
userRecs = model1.recommendForAllUsers(10)

movieRecs = model1.recommendForAllItems(10)

In [109]:
#Recommendations for all users from model1 approach

pd_userRecs = userRecs.toPandas()

pd_userRecs.head()

,userId,recommendations
0,15846,"[(3033, 4.981762886047363), (15557, 4.84286117..."
1,16386,"[(14648, 3.9512343406677246), (14283, 3.870496..."
2,128389,"[(15836, 4.1682209968566895), (15183, 4.123197..."
3,151871,"[(12293, 4.460720062255859), (3290, 4.37065649..."
4,205371,"[(2861, 4.651029586791992), (12952, 4.64796781..."


In [110]:
#Movie Recommendations for all users from model1 approach

pd_movieRecs = movieRecs.toPandas()

pd_movieRecs.head()

,movieId,recommendations
0,4935,"[(2635943, 4.867413520812988), (794999, 4.7332..."
1,11317,"[(1968542, 4.557005882263184), (642384, 4.5008..."
2,2659,"[(1419978, 4.514466762542725), (27061, 4.45776..."
3,13289,"[(2088784, 4.763981342315674), (1605728, 4.717..."
4,2366,"[(27061, 4.72322940826416), (844526, 4.6992330..."


### Approach 2, maxIter = 10 , regParam = 0.01

In [111]:
als = ALS(maxIter=10, regParam=0.01, userCol="userId", itemCol="movieId", ratingCol="ratings",
          coldStartStrategy="drop")
model2 = als.fit(df_training_join)



# Evaluate the model by computing the RMSE on the test data
predictions2 = model2.transform(df_testing_join)
evaluator = RegressionEvaluator(metricName="mse", labelCol="ratings",
                                predictionCol="prediction")
                                            
evaluator1 = RegressionEvaluator(metricName="rmse", labelCol="ratings",
                                predictionCol="prediction")



rmse = evaluator1.evaluate(predictions2)
mse = evaluator.evaluate(predictions2)

print("Mean squared error = " + str(mse))
print("Root-mean-square error = " + str(rmse))

Mean squared error = 0.7078012505083342
Root-mean-square error = 0.8413092478442955


### With the parameters maxIter = 10 , regParam = 0.01, Root-mean-square error is 0.84 and Mean squared error is 0.70

In [119]:
predictions2.show()

+-------+-------+-------+-------------+-------------+----------+
|movieId| userId|ratings|yearOfRelease|        title|prediction|
+-------+-------+-------+-------------+-------------+----------+
|   2366|1484561|    4.0|         2001|Just Visiting| 3.6417994|
|   2366| 397212|    2.0|         2001|Just Visiting| 1.8713087|
|   2366| 692235|    3.0|         2001|Just Visiting| 3.1074786|
|   2366|2552319|    3.0|         2001|Just Visiting| 2.8722143|
|   2366| 799826|    2.0|         2001|Just Visiting|  2.708126|
|   2366|1848661|    3.0|         2001|Just Visiting| 3.0266151|
|   2366|1870288|    2.0|         2001|Just Visiting|  3.767644|
|   2366|1889646|    4.0|         2001|Just Visiting| 3.8301644|
|   2366| 819822|    3.0|         2001|Just Visiting| 2.1934857|
|   2366| 125392|    3.0|         2001|Just Visiting| 2.8872359|
|   2366|2392538|    3.0|         2001|Just Visiting| 2.5809598|
|   2366| 742107|    5.0|         2001|Just Visiting| 3.1203933|
|   2366| 472083|    4.0|

In [120]:
userRecs2 = model2.recommendForAllUsers(10)

movieRecs2 = model2.recommendForAllItems(10)

In [121]:
userRecs2.show()

+------+--------------------+
|userId|     recommendations|
+------+--------------------+
| 15846|[[6991, 6.6622887...|
| 16386|[[14648, 4.029642...|
|128389|[[16576, 6.280640...|
|151871|[[7016, 5.4288054...|
|205371|[[14100, 7.861492...|
|263031|[[12864, 5.47393]...|
|279120|[[7527, 4.3905935...|
|284489|[[15809, 5.942722...|
|302344|[[14089, 5.194306...|
|331295|[[12864, 5.298617...|
|351141|[[15567, 5.410294...|
|376576|[[15480, 4.202807...|
|399735|[[6287, 4.3282576...|
|400440|[[11054, 6.136180...|
|426301|[[3359, 5.4057484...|
|434476|[[15567, 5.659684...|
|446160|[[6277, 6.3361664...|
|450125|[[8842, 6.34262],...|
|455334|[[14100, 5.306169...|
|496699|[[7016, 5.3026924...|
+------+--------------------+
only showing top 20 rows



In [122]:
movieRecs2.show()

+-------+--------------------+
|movieId|     recommendations|
+-------+--------------------+
|   4935|[[2573207, 6.4066...|
|  11317|[[1256990, 5.1191...|
|   2366|[[27061, 5.272477...|
|   2866|[[1606869, 5.7883...|
|   6336|[[1870572, 5.3850...|
|   6466|[[689215, 6.54576...|
|   9376|[[1606869, 8.3693...|
|  12046|[[973133, 6.84727...|
|   2659|[[428653, 5.48664...|
|  13289|[[1306404, 6.0939...|
|   2235|[[1306404, 4.8571...|
|  14075|[[1256990, 5.6500...|
|   4190|[[2302897, 7.1306...|
|   1522|[[973133, 6.38522...|
|  10462|[[761430, 4.472],...|
|   1896|[[2088784, 5.4494...|
|  11316|[[396985, 6.09421...|
|  16916|[[2100574, 5.2965...|
|   6482|[[825353, 5.51940...|
|  14324|[[61155, 6.605241...|
+-------+--------------------+
only showing top 20 rows



### From the above two model performances we can say the RMSE value is comparitively less when maxIter = 10 and regParam = 0.01. Hence this would be our best ALS model.

## Creating a new user for testing purpose and adding self rated movie ratings to our dataset 

In [74]:
user = 's3://netflixfinalvinay/myUser.txt'

myUser_df_schema = StructType(
  [StructField('movieId', IntegerType()),
   StructField('userId', IntegerType()),
   StructField('ratings', StringType())])

In [75]:
myUser_df = sqlContext.read.format('txt').options(inferSchema=True).schema(myUser_df_schema).csv(user)

myUser_df.show()

+-------+------+-------+
|movieId|userId|ratings|
+-------+------+-------+
|      8|     0|    4.0|
|     24|     0|    3.0|
|     41|     0|    4.0|
|     43|     0|    4.0|
|     61|     0|    2.0|
|     64|     0|    4.0|
|     48|     0|    4.0|
|     30|     0|    3.0|
|     12|     0|    4.0|
|     10|     0|    4.0|
+-------+------+-------+



In [76]:
myUser_df = myUser_df.join(df_movies,on=['movieId'],how='inner')

In [77]:
myUser_df.show()

+-------+------+-------+-------------+--------------------+
|movieId|userId|ratings|yearOfRelease|               title|
+-------+------+-------+-------------+--------------------+
|      8|     0|    4.0|         2004|What the #$*! Do ...|
|     10|     0|    4.0|         2001|             Fighter|
|     12|     0|    4.0|         1947|My Favorite Brunette|
|     24|     0|    3.0|         1981| My Bloody Valentine|
|     30|     0|    3.0|         2003|Something's Gotta...|
|     41|     0|    4.0|         2000|       Horror Vision|
|     43|     0|    4.0|         2000|      Silent Service|
|     48|     0|    4.0|         2001|      Justice League|
|     61|     0|    2.0|         1999|Ricky Martin: One...|
|     64|     0|    4.0|         2001|     Outside the Law|
+-------+------+-------+-------------+--------------------+



In [78]:
df_training_union = df_training_join.union(myUser_df)

df_testing_union = df_testing_join.union(myUser_df)

In [79]:
df_training_union.show()

+-------+-------+-------+-------------+--------------------+
|movieId| userId|ratings|yearOfRelease|               title|
+-------+-------+-------+-------------+--------------------+
|      8|1744889|    1.0|         2004|What the #$*! Do ...|
|      8|1395430|    2.0|         2004|What the #$*! Do ...|
|      8|1205593|    4.0|         2004|What the #$*! Do ...|
|      8|1488844|    4.0|         2004|What the #$*! Do ...|
|      8|1447354|    1.0|         2004|What the #$*! Do ...|
|      8| 306466|    4.0|         2004|What the #$*! Do ...|
|      8|1331154|    4.0|         2004|What the #$*! Do ...|
|      8|1818178|    3.0|         2004|What the #$*! Do ...|
|      8| 991725|    4.0|         2004|What the #$*! Do ...|
|      8|1987434|    4.0|         2004|What the #$*! Do ...|
|      8|1765381|    4.0|         2004|What the #$*! Do ...|
|      8| 433803|    3.0|         2004|What the #$*! Do ...|
|      8|1148143|    2.0|         2004|What the #$*! Do ...|
|      8|1174811|    5.0

In [81]:
UserID = 0
self = df_training_union.filter(df_training_union.userId == UserID)
self.show(30)

+-------+------+-------+-------------+--------------------+
|movieId|userId|ratings|yearOfRelease|               title|
+-------+------+-------+-------------+--------------------+
|      8|     0|    4.0|         2004|What the #$*! Do ...|
|     10|     0|    4.0|         2001|             Fighter|
|     12|     0|    4.0|         1947|My Favorite Brunette|
|     24|     0|    3.0|         1981| My Bloody Valentine|
|     30|     0|    3.0|         2003|Something's Gotta...|
|     41|     0|    4.0|         2000|       Horror Vision|
|     43|     0|    4.0|         2000|      Silent Service|
|     48|     0|    4.0|         2001|      Justice League|
|     61|     0|    2.0|         1999|Ricky Martin: One...|
|     64|     0|    4.0|         2001|     Outside the Law|
+-------+------+-------+-------------+--------------------+

